In [1]:
import pandas as pd
import geopandas as gp 
import json
from glob import glob
import numpy as np
import numpy as np
from tqdm import tqdm

In [2]:
ward_pop = pd.read_csv('../mappings/ward_pop.csv')

/work/sagarj/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
%store -r cityMap

no stored variable or alias cityMap


In [4]:
population = {}
for index, row in ward_pop.iterrows():
    population[row['Ward Code 1']] = float(row['All Ages'].replace(',',''))

In [5]:
df_LSOA_CITY = pd.read_csv('../mappings/lower_layer_super_output_area_2011_to_major_towns_and_cities_december_2015_lookup_in_england_and_wales.csv')

In [6]:
df_LSOA_WARD = pd.read_csv('../mappings/lower_layer_super_output_area_(2011)_to_ward_(2015)_lookup_in_england_and_wales.csv')

In [7]:
df_LSOA_WARD.head()

,LSOA11CD,LSOA11NM,WD15CD,WD15NM,WD15NMW,LAD15CD,LAD15NM,FID
0,E01000104,Barking and Dagenham 014B,E05000041,Village,,E09000002,Barking and Dagenham,1
1,E01000105,Barking and Dagenham 014C,E05000041,Village,,E09000002,Barking and Dagenham,2
2,E01000106,Barking and Dagenham 014D,E05000041,Village,,E09000002,Barking and Dagenham,3
3,E01000107,Barking and Dagenham 010C,E05000041,Village,,E09000002,Barking and Dagenham,4
4,E01000108,Barking and Dagenham 010D,E05000041,Village,,E09000002,Barking and Dagenham,5


In [8]:
df_LSOA_CITY.head()

,LSOA11CD,LSOA11NM,TCITY15CD,TCITY15NM,FID
0,E01002351,Havering 016C,J01000055,London,2001
1,E01002352,Havering 016D,J01000055,London,2002
2,E01002100,Haringey 008B,J01000055,London,2003
3,E01002301,Havering 003A,J01000055,London,2004
4,E01002353,Havering 013B,J01000055,London,2005


In [9]:
mega_df = pd.merge(df_LSOA_WARD,df_LSOA_CITY,left_on='LSOA11CD' , right_on='LSOA11CD')

In [10]:
mega_df.head()

,LSOA11CD,LSOA11NM_x,WD15CD,WD15NM,WD15NMW,LAD15CD,LAD15NM,FID_x,LSOA11NM_y,TCITY15CD,TCITY15NM,FID_y
0,E01000104,Barking and Dagenham 014B,E05000041,Village,,E09000002,Barking and Dagenham,1,Barking and Dagenham 014B,J01000055,London,151
1,E01000105,Barking and Dagenham 014C,E05000041,Village,,E09000002,Barking and Dagenham,2,Barking and Dagenham 014C,J01000055,London,152
2,E01000106,Barking and Dagenham 014D,E05000041,Village,,E09000002,Barking and Dagenham,3,Barking and Dagenham 014D,J01000055,London,153
3,E01000107,Barking and Dagenham 010C,E05000041,Village,,E09000002,Barking and Dagenham,4,Barking and Dagenham 010C,J01000055,London,154
4,E01000108,Barking and Dagenham 010D,E05000041,Village,,E09000002,Barking and Dagenham,5,Barking and Dagenham 010D,J01000055,London,155


In [11]:
sub_data = mega_df[['LSOA11CD','WD15CD','WD15NM','LAD15CD','LAD15NM','TCITY15NM']].copy()

In [12]:
sub_data.rename(columns={'LAD15NM':'LocalAuthorityName' , 'TCITY15NM':'CityName'},inplace=True)

In [13]:
sub_data.CityName = sub_data.CityName.fillna('')

In [14]:
sub_data

,LSOA11CD,WD15CD,WD15NM,LAD15CD,LocalAuthorityName,CityName
0,E01000104,E05000041,Village,E09000002,Barking and Dagenham,London
1,E01000105,E05000041,Village,E09000002,Barking and Dagenham,London
2,E01000106,E05000041,Village,E09000002,Barking and Dagenham,London
3,E01000107,E05000041,Village,E09000002,Barking and Dagenham,London
4,E01000108,E05000041,Village,E09000002,Barking and Dagenham,London
...,...,...,...,...,...,...
34748,W01001911,W05000635,Castleland,W06000014,Vale of Glamorgan,
34749,W01001860,W05000877,Riverside,W06000015,Cardiff,Cardiff
34750,W01001753,W05000861,Fairwater,W06000015,Cardiff,Cardiff
34751,W01001754,W05000861,Fairwater,W06000015,Cardiff,Cardiff


In [15]:
pop = pd.read_csv('../mappings/2015_england_pop.csv',thousands=',')

In [16]:
pop.head(n=3)

,Area Codes,All Ages
0,E06000047,519695
1,E01020634,1521
2,E01020635,1307


In [17]:
final_sub = pd.merge(sub_data,pop,left_on='LSOA11CD',right_on='Area Codes')

In [18]:
len(final_sub)

34753

In [19]:
final_sub.to_csv('../data_prep/Wards.csv')

In [20]:
# LSOA_dist = json.load(open('../mappings/GP_LSOA_PATIENTSDIST.json','rb'))

In [21]:
# LSOA_dist_2021 = json.load(open('mappings/GP_LSOA_PATIENTSDIST_2021.json','rb'))

In [22]:
# LSOA_dist['Y02811'].values()

In [31]:
#This file comes from here https://digital.nhs.uk/data-and-information/publications/statistical/patients-registered-at-a-gp-practice.
# LSOA_dist_new = pd.read_csv('mappings/gp-reg-pat-prac-lsoa-all.csv')
LSOA_dist_new = pd.read_csv('../mappings/gp-reg-patients-04-2014-totals-lsoa-alt.csv')

In [32]:
LSOA_dist_new.columns

Index(['PRACTICE_CODE', 'ORG_NAME', 'LSOA_CODE', 'Male Patients',
       'Female Patients', 'All Patients'],
      dtype='object')

In [33]:
LSOA_dist_new.head()

,PRACTICE_CODE,ORG_NAME,LSOA_CODE,Male Patients,Female Patients,All Patients
0,A81001,THE DENSHAM SURGERY,E01012187,0,4,4
1,A81001,THE DENSHAM SURGERY,E01012188,43,48,91
2,A81001,THE DENSHAM SURGERY,E01012189,26,32,58
3,A81001,THE DENSHAM SURGERY,E01012190,68,86,154
4,A81001,THE DENSHAM SURGERY,E01012191,72,90,162


In [58]:
LSOA_dist_2013 = {}
for name , group in tqdm(LSOA_dist_new.groupby('PRACTICE_CODE')):
    LSOA_dist_2013[name] = {}
    total = sum(group['All Patients'])
    LSOA_dist_2013[name]['Patient_registry_LSOA'] = {}
    LSOA_dist_2013[name]['Total_patients'] = total
    LSOA_dist_2013[name]['GP_Name'] = group['ORG_NAME'].iloc[0]
    LSOA_dist_2013[name]['GP_Postcode'] = ''
    LSOA_dist_2013[name]['GP_Address'] = ''
    for index , row in group.iterrows():
        LSOA_dist_2013[name]['Patient_registry_LSOA'][row['LSOA_CODE']] = float(row['All Patients'])

100%|██████████| 7997/7997 [01:15<00:00, 105.28it/s]


In [59]:
# LSOA_dist_2021 = {}
# for name , group in tqdm(LSOA_dist_new.groupby('PRACTICE_CODE')):
#     LSOA_dist_2021[name] = {}
#     total = sum(group['Number of Patients'])
#     LSOA_dist_2021[name]['Patient_registry_LSOA'] = {}
#     LSOA_dist_2021[name]['Total_patients'] = total
#     LSOA_dist_2021[name]['GP_Name'] = group['PRACTICE_NAME'].iloc[0]
#     LSOA_dist_2021[name]['GP_Postcode'] = ''
#     LSOA_dist_2021[name]['GP_Address'] = ''
#     for index , row in group.iterrows():
#         LSOA_dist_2021[name]['Patient_registry_LSOA'][row['LSOA_CODE']] = float(row['Number of Patients'])

In [60]:
sum(LSOA_dist_new['All Patients'])

56442636

In [61]:
# LSOA_dist_new_codes = LSOA_dist_2013.keys()
# LSOA_dist_codes = LSOA_dist.keys()
# print(len(set(LSOA_dist_new_codes).intersection(LSOA_dist_codes))) , print(len(LSOA_dist_new_codes)) , print(len(LSOA_dist_codes))

In [63]:
presc1 = pd.read_csv('/datasets/sagarj/NHS_serialized_data/201401.gz',compression='gzip')

/work/sagarj/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
# presc2 = pd.read_csv('/datasets/sagarj/NHS_serialized_data/202101.gz',compression='gzip')

In [65]:
# presc2.head(n=2)

In [66]:
presc1.head(n=2)

,Unnamed: 0,YEAR_MONTH,REGIONAL_OFFICE_NAME,REGIONAL_OFFICE_CODE,STP_NAME,STP_CODE,PCO_NAME,PCO_CODE,PRACTICE_NAME,PRACTICE_CODE,...,NIC,ACTUAL_COST,UNIDENTIFIED,15,16,17,18,19,20,21
0,0,202102,SOUTH EAST,Y59,"BUCKS, OXFORDSHIRE & BERKSHIRE WEST STP",QU9,OXFORDSHIRE CCG,10Q00,MORLAND HOUSE SURGERY,K84014,...,6.66,6.21560,N,0.0,130201100,500.0,1.0,500.0,BB,0.0
1,1,202102,SOUTH EAST,Y59,"BUCKS, OXFORDSHIRE & BERKSHIRE WEST STP",QU9,OXFORDSHIRE CCG,10Q00,MORLAND HOUSE SURGERY,K84014,...,7.10,6.62542,N,0.0,130201100,400.0,1.0,400.0,BB,0.0


In [67]:
# filter_new_presc_new_codes = presc2[presc2['PRACTICE_CODE'].isin(LSOA_dist_new_codes)]
# filter_new_presc_old_codes = presc2[presc2['PRACTICE_CODE'].isin(LSOA_dist_codes)]
# filter_old_presc_new_codes = presc1[presc1['PRACTICE_CODE'].isin(LSOA_dist_new_codes)]
filter_old_presc_old_codes = presc1[presc1['PRACTICE_CODE'].isin(LSOA_dist_2013)]

In [68]:
# filter_old_presc_old_codes['2'].nunique() , filter_old_presc_new_codes['2'].nunique() 

In [69]:
# filter_new_presc_old_codes['PRACTICE_CODE'].nunique() , filter_new_presc_new_codes['PRACTICE_CODE'].nunique() 

In [70]:
# presc2.columns

In [71]:
presc1[['PRACTICE_CODE' , 'ADDRESS_1' , 'ADDRESS_2' , 'ADDRESS_3', 'ADDRESS_4', 'POSTCODE']]

,PRACTICE_CODE,ADDRESS_1,ADDRESS_2,ADDRESS_3,ADDRESS_4,POSTCODE
0,K84014,MORLAND HOUSE SURGERY,"LONDON ROAD, WHEATLEY",OXFORD,OXFORDSHIRE,OX33 1YJ
1,K84014,MORLAND HOUSE SURGERY,"LONDON ROAD, WHEATLEY",OXFORD,OXFORDSHIRE,OX33 1YJ
2,K84014,MORLAND HOUSE SURGERY,"LONDON ROAD, WHEATLEY",OXFORD,OXFORDSHIRE,OX33 1YJ
3,K84014,MORLAND HOUSE SURGERY,"LONDON ROAD, WHEATLEY",OXFORD,OXFORDSHIRE,OX33 1YJ
4,K84014,MORLAND HOUSE SURGERY,"LONDON ROAD, WHEATLEY",OXFORD,OXFORDSHIRE,OX33 1YJ
...,...,...,...,...,...,...
13142278,M85156,RIVER BROOK MEDICAL CTRE,3 RIVER BROOK DRIVE,STIRCHLEY,BIRMINGHAM,B30 2SH
13142279,M85156,RIVER BROOK MEDICAL CTRE,3 RIVER BROOK DRIVE,STIRCHLEY,BIRMINGHAM,B30 2SH
13142280,M85156,RIVER BROOK MEDICAL CTRE,3 RIVER BROOK DRIVE,STIRCHLEY,BIRMINGHAM,B30 2SH
13142281,M85156,RIVER BROOK MEDICAL CTRE,3 RIVER BROOK DRIVE,STIRCHLEY,BIRMINGHAM,B30 2SH


In [72]:
GP_address = {}
for name , group in tqdm(presc1.groupby('PRACTICE_CODE')):
    GP_address[name] = {}
    GP_address[name]['GP_Postcode'] = group['POSTCODE'].iloc[0]
    GP_address[name]['GP_Address'] = ' '.join([group['ADDRESS_1'].iloc[0] ,group['ADDRESS_2'].iloc[0] ,group['ADDRESS_3'].iloc[0] ,group['ADDRESS_4'].iloc[0] ])

100%|██████████| 5106/5106 [00:05<00:00, 1008.73it/s]


In [73]:
for k in tqdm(GP_address):
    if k in LSOA_dist_2013:
        LSOA_dist_2013[k]['GP_Address'] = GP_address[k]['GP_Address']
        LSOA_dist_2013[k]['GP_Postcode'] = GP_address[k]['GP_Postcode']

100%|██████████| 5106/5106 [00:00<00:00, 896221.80it/s]


In [74]:
LSOA_dist_2013['N83031']

{'Patient_registry_LSOA': {'E01006419': 106.0,
  'E01006420': 135.0,
  'E01006421': 64.0,
  'E01006422': 78.0,
  'E01006423': 105.0,
  'E01006424': 3.0,
  'E01006425': 45.0,
  'E01006426': 206.0,
  'E01006427': 147.0,
  'E01006428': 227.0,
  'E01006429': 208.0,
  'E01006430': 42.0,
  'E01006431': 185.0,
  'E01006432': 412.0,
  'E01006433': 41.0,
  'E01006441': 4.0,
  'E01006443': 1.0,
  'E01006463': 1.0,
  'E01006483': 3.0,
  'E01006503': 1.0,
  'E01006505': 5.0,
  'E01006521': 1.0,
  'E01006524': 1.0,
  'E01006530': 2.0,
  'E01006531': 6.0,
  'E01006532': 4.0,
  'E01006533': 6.0,
  'E01006535': 2.0,
  'E01006536': 3.0,
  'E01006537': 30.0,
  'E01006538': 10.0,
  'E01006540': 1.0,
  'E01006551': 1.0,
  'E01006560': 1.0,
  'E01006570': 4.0,
  'E01006575': 1.0,
  'E01006576': 1.0,
  'E01006581': 3.0,
  'E01006589': 2.0,
  'E01006590': 2.0,
  'E01006593': 1.0,
  'E01006594': 1.0,
  'E01006682': 3.0,
  'E01006689': 3.0,
  'E01006706': 1.0,
  'E01006708': 11.0,
  'E01006709': 7.0,
  'E01006

In [75]:
len(LSOA_dist_2013.keys())

7997

In [82]:
json.dump(LSOA_dist_2013,open('../mappings/GPs_2013.json','w'))

In [79]:
len(LSOA_dist_2013.keys())

7997

In [80]:
LSOA_GP_dict_df = {'GP_ID' : [] , 'LSOA2011' : [] , 'Patient_registry_fraction' : [] , 'Total_patients' : []}
for gpid in LSOA_dist_2013:
    total = LSOA_dist_2013[gpid]['Total_patients']
    for lsoa in LSOA_dist_2013[gpid]['Patient_registry_LSOA']:
        LSOA_GP_dict_df['GP_ID'].append(gpid)
        LSOA_GP_dict_df['LSOA2011'].append(lsoa)
        LSOA_GP_dict_df['Patient_registry_fraction'].append(float(LSOA_dist_2013[gpid]['Patient_registry_LSOA'][lsoa])/float(total))
        LSOA_GP_dict_df['Total_patients'].append(total)
LSOA_GP_df = pd.DataFrame.from_dict(LSOA_GP_dict_df)
    

In [81]:
sum(LSOA_GP_df[LSOA_GP_df['GP_ID']=='A81001']['Patient_registry_fraction'])

1.0

In [83]:
LSOA_GP_df.to_csv('../mappings/GP_LSOA_weights_2013.csv')

In [84]:
len(LSOA_GP_df)

729516

In [85]:
LSOA_GP_df[LSOA_GP_df['GP_ID']=='N83031'].head()

,GP_ID,LSOA2011,Patient_registry_fraction,Total_patients
610077,N83031,E01006419,0.047769,2219
610078,N83031,E01006420,0.060838,2219
610079,N83031,E01006421,0.028842,2219
610080,N83031,E01006422,0.035151,2219
610081,N83031,E01006423,0.047319,2219
